In [28]:
import os
import win32com.client
import pythoncom
import datetime, time# for sleep
import sys
import pickle
import os.path
from os import path
import pandas as pd # command 로 시작할 때는 모듈 import 위치고려
from datetime import timedelta
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from apiclient.http import MediaFileUpload

import base64
from github import Github
from github import InputGitTreeElement

from init_Stock import *

df = pd.DataFrame(Target_Frame)

##################################################
# 최초 접속
##################################################

def __Init_Conn() :

    class XASessionEventHandler:
        
        login_state = 0
        def OnLogin(self, code, msg):

            if code == "0000":
                print("로그인 성공")
                XASessionEventHandler.login_state = 1
            else:
                print("로그인 실패")

    instXASession = win32com.client.DispatchWithEvents("XA_Session.XASession", XASessionEventHandler)

    # 접속할 서버의 기본 주소는 'hts.ebestsec.co.kr'인데 모의 투자인 경우에는 'demo.ebestsec.co.kr'을 사용
#     print("Server_Name : %s" % Server_Name)
    instXASession.ConnectServer(Server_Name, 20001)
    instXASession.Login(id, passwd, cert_passwd, 0, 0)

    while XASessionEventHandler.login_state == 0:
        pythoncom.PumpWaitingMessages()

    num_account = instXASession.GetAccountListCount()

##################################################
# 현재가 확인
##################################################

def __Price_Check(Target_Expcode) :

    print("# Price_Check Start : %s, hcode : %s" % (datetime.datetime.today().strftime("%H:%M:%S"), Target_Expcode))

    class XAQueryEventHandlerT1101:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT1101.query_state = 1
            
        def OnReceiveMessage(self, error, code, message):
            print("OnreceiveMessage", error, code, message)

    instXAQueryT1101 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT1101)
    instXAQueryT1101.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t1101.res"
    instXAQueryT1101.SetFieldData("t1101InBlock", "shcode", 0, Target_Expcode)
    
    instXAQueryT1101.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT1101.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()

    hname = instXAQueryT1101.GetFieldData("t1101OutBlock", "hname", 0)
    price = instXAQueryT1101.GetFieldData("t1101OutBlock", "price", 0)
    diff = instXAQueryT1101.GetFieldData("t1101OutBlock", "diff", 0) # 등락율
    bidho2 = instXAQueryT1101.GetFieldData("t1101OutBlock", "bidho2", 0) # 매수호가2
    offerho2 = instXAQueryT1101.GetFieldData("t1101OutBlock", "offerho2", 0) # 매도호가2
    print("종목 : %s, 가격 : %s, 매수호가2 : %s, 매도호가2 : %s" % (hname, price, bidho2, offerho2))
    print("# Price_Check End : %s, hcode : %s" % (datetime.datetime.today().strftime("%H:%M:%S"), Target_Expcode))
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Main_Status_History"+_Name+".txt"
    output = open(File_Name, "a")
    output.write("# %s : ___Price_Check ---> 종목 : %s, 가격 : %s, 매수호가2 : %s, 매도호가2 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), hname, price, bidho2, offerho2))
    output.close() # ---> close 가 되어야 write 처리가 완료된다

    instXAQueryT1101.close()

    return price, bidho2, diff # 가격, 매수호가2, 등락율

##################################################
# 우선주 괴리율 확인 (300개)
##################################################

def __Preferred_Stock() :
    
    Long_Term_Dic = {}
    
#     for x in range(0,2) :
#         print("string{0}".format(x))

    Daily_Change_Pref_a, Daily_Change_Pref_b = {}, {}

    for expcode in Preferred_Expcode_List_1 :
#         print("df_{0}".format(expcode)) # --->  for문안에서 변수 생성 ***** 중요
        time.sleep(1)
        ##################################################
        # 주식 현재가 호가 조회
        class XAQueryEventHandlerT1305:
            query_state = 0

            def OnReceiveData(self, code):
                XAQueryEventHandlerT1305.query_state = 1

            def OnReceiveMessage(self, error, code, message):
                print("OnreceiveMessage", error, code, message)

        ##################################################
        instXAQueryT1305 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT1305)
        instXAQueryT1305.ResFileName = "C:\\eBEST\\xingAPI\\Res\\T1305.res"
        instXAQueryT1305.SetFieldData("T1305InBlock", "shcode", 0, expcode)
        instXAQueryT1305.SetFieldData("T1305InBlock", "dwmcode", 0, 1)
        instXAQueryT1305.SetFieldData("T1305InBlock", "cnt", 0, 365)

        instXAQueryT1305.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

        while XAQueryEventHandlerT1305.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
            pythoncom.PumpWaitingMessages()

        count = instXAQueryT1305.GetBlockCount("t1305OutBlock1")

        date_list, open_list, close_list, change_list, diff_list = [], [], [], [], []
        df_Daily_Change_Pref = pd.DataFrame()
        for i in range(count):
            date = instXAQueryT1305.GetFieldData("T1305OutBlock1", "date", i)
            close = instXAQueryT1305.GetFieldData("T1305OutBlock1", "close", i)
            change = instXAQueryT1305.GetFieldData("T1305OutBlock1", "change", i) # 전일대비
            diff = instXAQueryT1305.GetFieldData("T1305OutBlock1", "diff", i) # 등락율
            date_list.append(date)
            close_list.append(close)
            change_list.append(change)
            diff_list.append(diff)

        Daily_Change_Pref = {'date' : date_list, 'close' : close_list, 'change' : change_list, 'diff' : diff_list}
        Daily_Change_Pref["diff"] = pd.to_numeric(Daily_Change_Pref["diff"]) # 변환 안해주면 ---> TypeError: unsupported operand type(s) for -: 'list' and 'list'
        Daily_Change_Pref["close"] = pd.to_numeric(Daily_Change_Pref["close"]) # 변환 안해주면 ---> TypeError: unsupported operand type(s) for -: 'list' and 'list'
        
        a_has_items = bool(Daily_Change_Pref_a)
        b_has_items = bool(Daily_Change_Pref_b)
        if not a_has_items :
#             print("A is empty : %s" % a_has_items)
            Daily_Change_Pref_a = Daily_Change_Pref
        elif not b_has_items :
#             print("B is empty : %s" % b_has_items)
            Daily_Change_Pref_b = Daily_Change_Pref
        else :
            print("ELSE Statement")
        
        instXAQueryT1305.close()

    # df_Daily_Change_Pref_a = pd.DataFrame()
    df_Daily_Change_Pref_a = pd.DataFrame(Daily_Change_Pref_a) # 보통주
    df_Daily_Change_Pref_b = pd.DataFrame(Daily_Change_Pref_b) # 우선주
    
    df_Daily_Change_Pref_a["date2"] = df_Daily_Change_Pref_b["date"] # 우선주 날짜
    df_Daily_Change_Pref_a["close2"] = df_Daily_Change_Pref_b["close"] # 우선주 종가
    df_Daily_Change_Pref_a["change2"] = df_Daily_Change_Pref_b["change"] # 우선주 전일대비
    df_Daily_Change_Pref_a["diff2"] = df_Daily_Change_Pref_b["diff"] # 우선주 등락율
    df_Daily_Change_Pref_a["Diff_Close"] = round((df_Daily_Change_Pref_a["close"] - df_Daily_Change_Pref_a["close2"]) / df_Daily_Change_Pref_a["close2"] * 100, 1)
    
    # print(df_Daily_Change_Pref_a)
    # df_Daily_Change = df_Daily_Change[df_Daily_Change["diff"] < 0] # ---> minus 인 날만 list
    Min_Diff = round(df_Daily_Change_Pref_a[df_Daily_Change_Pref_a["Diff_Close"]==df_Daily_Change_Pref_a["Diff_Close"].min()], 1) # ---> minus 인 날 중 최대 min은 제외
    Max_Diff = round(df_Daily_Change_Pref_a[df_Daily_Change_Pref_a["Diff_Close"]==df_Daily_Change_Pref_a["Diff_Close"].max()], 1) # ---> minus 인 날 중 최대 min은 제외

    preferred_diff = round(df_Daily_Change_Pref_a["Diff_Close"].mean(), 1)
    preferred_diff_Max = round(df_Daily_Change_Pref_a["Diff_Close"].max(), 1)

    # 평균 1.3이상으로하면 max값보다 상회 ---> 값 평균과 max가 그리 차이가 안남
    # max와 평균값의 평균값으로Perferred_Diff_expect = round(preferred_diff * 1.15, 1)
    Perferred_Diff_expect_1 = round(preferred_diff * 1.15, 1)
    Perferred_Diff_expect_2 = round((preferred_diff + preferred_diff_Max) / 2, 1)

    print("매수 계획값 1 : %s" % Perferred_Diff_expect_1)
    print("매수 계획값 2 : %s" % Perferred_Diff_expect_2)
    
    return Perferred_Diff_expect_2

##################################################
# 보통주, 우선주 현재값 비교
##################################################

def __Preferred_Check_Price() :

    Preferred_Price_List_1 = ''
    Preferred_Price_List_2 = ''
    
    for t_expcode in Preferred_Expcode_List_1 : # init_Stock 의 우선주 종목이 보통주, 우선주 순서
#         print("종목명 : %s" % t_expcode)
        Stock_Price, Price_Check_HIDHO2, diff = __Price_Check(t_expcode)
        
        if not bool(Preferred_Price_List_1) :
#             print("넣자")
            Preferred_Price_List_1 = Stock_Price
        elif not bool(Preferred_Price_List_2) :
#             print("넣자니깐")
            Preferred_Price_List_2 = Stock_Price
        else :
            print("보통주, 우선주 현재값 비교안되네")

    Preferred_Price_List_1 = int(Preferred_Price_List_1) # 보통주
    Preferred_Price_List_2 = int(Preferred_Price_List_2) # 우선주
    
    Perferred_Diff_real = round((Preferred_Price_List_1 - Preferred_Price_List_2) / Preferred_Price_List_2 * 100, 1)
    
    return Perferred_Diff_real
        
##################################################
# Main Loop
##################################################

TEST_Try = 1 # TEST 조건이 1이면서 토요일이나 일요일이면 그냥 나간다
# TEST_Try = 0

if __name__ == '__main__' :
    
        __Init_Conn()
        Plan_Diff = __Preferred_Stock()
        Real_Diff = __Preferred_Check_Price()
        
        if (Plan_Diff < Real_Diff) :
            print("Plan : %s, Real : %s ---> 매수" % (Plan_Diff, Real_Diff))
        else :
            print("Plan : %s, Real : %s ---> 대기" % (Plan_Diff, Real_Diff))
            
        print("# 모든 실행 종료")
        

로그인 성공
OnreceiveMessage 0 00000 조회완료
OnreceiveMessage 0 00000 조회완료
매수 계획값 1 : 91.0
매수 계획값 2 : 97.5
# Price_Check Start : 12:40:40, hcode : 005380
OnreceiveMessage 0 00000 조회완료
종목 : 현대차, 가격 : 177000, 매수호가2 : 176500, 매도호가2 : 178000
# Price_Check End : 12:40:40, hcode : 005380
# Price_Check Start : 12:40:40, hcode : 005385
OnreceiveMessage 0 00000 조회완료
종목 : 현대차우, 가격 : 85300, 매수호가2 : 85200, 매도호가2 : 85500
# Price_Check End : 12:40:40, hcode : 005385
Plan : 91.0, Real : 107.5 ---> 매수
# 모든 실행 종료


In [2]:
a = 178000
b = 85500
c = (a-b) / a * 100
print(c)

Preferred_Expcode_List_1 = ["005380", "005387"] # 현대차 보통주, 현대차2B우선주 순서

print(Preferred_Expcode_List_1[0])
print(Preferred_Expcode_List_1[1])

51.96629213483146
005380
005387
